In [1]:
# Instalar pyodbc
# import sys
# !{sys.executable} -m pip install pyodbc
# import sys
# !{sys.executable} -m pip install openpyxl
# !{sys.executable} -m pip install --upgrade seaborn
# !{sys.executable} -m pip install --upgrade matplotlib

In [2]:
#Librería pyodbc
import pyodbc
import pandas as pd
import numpy as np
import openpyxl

# Conexión al Access
# Creo función 
def py_conn():
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=C:\\Users\\MEC\\OneDrive - BCRA\\General\\GH\\clustering.accdb;'
        )
    conn = pyodbc.connect(conn_str)

    return conn


def py_conn_tables(cursor):
    for table_info in cursor.tables(tableType='TABLE'):
        print(table_info.table_name)

In [3]:
# Consulta tabla indicadores
conn = py_conn()
data = pd.read_sql_query("SELECT * FROM DATA", conn)
conn.close()
# Veo la tabla que me trajo
print(data)

C:\Users\MEC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


        EENTIDAD CODIND     FECHAS         IND
0          00262    C31 2019-03-01  702.188401
1          00262    C31 2019-04-01  689.433633
2          00262    C31 2019-05-01  660.587483
3          00262    C31 2019-06-01  606.182058
4          00262    C31 2019-07-01  540.094705
...          ...    ...        ...         ...
2278404        8    RG9 2021-07-01   -7.141944
2278405        8    RG9 2021-08-01   -7.100442
2278406        8    RG9 2021-09-01   -7.357433
2278407        8    RG9 2021-10-01   -7.415377
2278408        8    RG9 2021-11-01   -7.303206

[2278409 rows x 4 columns]


In [4]:
# Elimino sistema, ghs y gh8
gh = ['1', '2', '3', '4', '5', '6', '7', '8', '00', '10', '11',
		'44099', '44098', '44096', '44095', '44088', '44093', '44092', '44090', '00340', '44059', '44094'
		]
# data= data[~data['EENTIDAD'].isin(gh)]

In [5]:
# Elimino indicadores que no necesito
ind_cod = [
          'RG2', 'RG1', 'E2', 'E1', 'RAPM',
          'A6', 'AG1', 'A2', 'A21',
          'L1', 'L8',
          'C1', 'C14', 'C2', 
          'LS1', 'LS2', 'MS1'
		]
# data = data[data['CODIND'].isin(ind_cod)]


In [6]:
# Datos desde 2013
data = data[data['FECHAS'].dt.strftime('%Y') >= '2013']
print(data)

        EENTIDAD CODIND     FECHAS         IND
0          00262    C31 2019-03-01  702.188401
1          00262    C31 2019-04-01  689.433633
2          00262    C31 2019-05-01  660.587483
3          00262    C31 2019-06-01  606.182058
4          00262    C31 2019-07-01  540.094705
...          ...    ...        ...         ...
2278404        8    RG9 2021-07-01   -7.141944
2278405        8    RG9 2021-08-01   -7.100442
2278406        8    RG9 2021-09-01   -7.357433
2278407        8    RG9 2021-10-01   -7.415377
2278408        8    RG9 2021-11-01   -7.303206

[1122847 rows x 4 columns]


In [7]:
# Elimino entidades NO vigentes
conn = py_conn()
entidad = pd.read_sql_query("SELECT * FROM ENTIDAD", conn)
conn.close()
vigente = entidad['EENTIDAD'][entidad['VIGENTE']==True]
# print(vigente)
data = data[data['EENTIDAD'].isin(vigente)]

C:\Users\MEC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
# Cuantos valores nulos hay
print(f'Cantidad de valores nulos por columna:\n{data.isna().sum()}') # Chequear RAPM

Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
FECHAS      0
IND         0
dtype: int64


In [9]:
# Conjunto de indicadores con valore no nulos
data = data[data.isna()==False]

# Creo la columna de año, saco promedio de cada diciembre
data['%Y'] = data['FECHAS'].dt.strftime('%Y')
data_dic_mean = data
data_dic_mean['IND'].replace(0, np.nan)
data_dic_mean = data_dic_mean.groupby(['EENTIDAD', 'CODIND', '%Y']).mean()

# Tabla con promedios totales
data_mean = data.replace(0, np.nan)
data_mean = data.drop(columns=['FECHAS']).groupby(['EENTIDAD', 'CODIND'], as_index=False).mean()
# data_mean = data_mean.replace(0, np.nan)
# print(data_mean)

print(f'Cantidad de valores nulos por columna:\n{data.isna().sum()}')
print(f'Cantidad de valores nulos por columna:\n{data_dic_mean.isna().sum()}')
print(f'Cantidad de valores nulos por columna:\n{data_mean.isna().sum()}')

data_mean = data_mean.reset_index()
data_dic_mean = data_dic_mean.reset_index()

Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
FECHAS      0
IND         0
%Y          0
dtype: int64
Cantidad de valores nulos por columna:
IND    0
dtype: int64
Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
IND         0
dtype: int64


In [10]:
data_mean_pivot = data_mean.pivot(index='EENTIDAD', columns='CODIND', values='IND').reset_index()
data_mean_pivot = data_mean_pivot.set_index('EENTIDAD')
print(data_mean_pivot.isna().sum())
data_mean_pivot.fillna(0.0)

CODIND
A1     0
A10    3
A11    3
A12    4
A13    8
      ..
RG5    0
RG6    0
RG7    0
RG8    0
RG9    0
Length: 139, dtype: int64


CODIND,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,...,RG18,RG1II,RG2,RG3,RG4,RG5,RG6,RG7,RG8,RG9
EENTIDAD,,,,,,,,,,,,,,,,,,,,,
00007,49.933370,3.526290,2.244626,62.190439,37.809562,-145.708886,-145.063669,3.698669,1.345622,18.010170,...,29.558509,3.585026,9.679914,-1.761804,3.042471,-6.132578,0.857366,0.000000,-1.414985,-0.707282
00011,55.658614,5.171001,3.028198,68.897075,31.102926,-212.557821,-207.742913,0.748891,4.954244,15.583017,...,65.135646,2.401787,7.111510,-0.898627,1.014080,-3.696043,0.551503,0.019641,-1.271060,-0.219491
00014,808.413597,4.486518,3.809970,53.095038,46.904963,-110.652090,-110.620276,4.193170,3.575293,18.146326,...,51.954680,2.244306,9.492312,-1.184617,2.375015,-8.547280,-0.123671,0.062687,0.000000,-0.253379
00015,41.162125,2.818136,1.959678,60.361521,39.638479,-132.790015,-131.532380,3.620750,0.872613,16.989724,...,36.360331,4.045053,13.117535,-1.526625,3.105206,-9.856044,1.003356,0.000000,-1.735480,-0.531748
00016,32.736063,2.217835,0.795972,82.728993,22.814789,-337.880191,-332.270500,1.377841,0.474179,27.565689,...,24.726901,12.249068,21.872935,-0.406432,3.162584,-7.552817,0.313192,0.000000,-5.230872,-1.726051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44099,21.149737,1.516424,1.547751,30.677568,69.322432,-86.986211,-86.986211,1.476767,1.703824,0.054076,...,0.000000,1.435789,7.951637,-0.509361,1.709257,-5.748181,0.578041,0.000000,-0.835494,-1.288441
45030,74.767017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.446904,-130.840524,30.557629,0.000000,-0.188476,-151.446985,0.003294,0.000000,29.568561,-39.357873
45056,81.562652,10.708812,17.542401,4.375069,95.624932,-59.735807,-59.735807,17.415372,22.961844,14.445231,...,26.836571,2.616166,23.612765,-5.854507,4.566328,-25.352419,3.810393,0.000000,0.050160,-0.218056


In [153]:
# Create correlation matrix
corr_matrix = data_mean_pivot.corr().abs()

with pd.ExcelWriter('output/data_corr.xlsx') as writer:
    
    for i in corr_matrix:
        ind_corr_matrix = corr_matrix[i].sort_values(ascending=False)
        ind_corr_matrix.to_excel(writer, sheet_name=i)

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    corr_matrix.to_excel(writer, sheet_name='upper_corr')

    upper_high_corr = upper[upper>=0.75]
    corr_ind_list = [(ind, upper_high_corr.index[upper_high_corr[ind]==el][0], el) for ind in upper_high_corr.columns for el in upper_high_corr[ind] if el>=0]
    corr_ind_df = pd.DataFrame(corr_ind_list, columns=['ind1', 'ind2', 'corr']).sort_values(by='ind1')

    corr_ind_df.to_excel(writer, sheet_name='high_corr_list')

C:\Users\MEC\AppData\Local\Temp\ipykernel_5548\3915735876.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = data_mean_pivot.corr()
sns.heatmap(cor) # , annot=True, cmap=plt.cm.Reds)
plt.savefig(f'graph/heatmap_corr.jpg')
plt.clf() # Clean parirplot figure from sns

<Figure size 864x720 with 0 Axes>

In [13]:
'''# print(corr_matrix)
for ind in corr_matrix:
    print(corr_matrix[ind])
    # print(ind)
    # print([i for i in enumerate(corr_matrix) if corr_matrix[ind])
    for i in corr_matrix[ind]>0.9:
        if i:
            print(i)'''

'# print(corr_matrix)\nfor ind in corr_matrix:\n    print(corr_matrix[ind])\n    # print(ind)\n    # print([i for i in enumerate(corr_matrix) if corr_matrix[ind])\n    for i in corr_matrix[ind]>0.9:\n        if i:\n            print(i)'